## Introduction:

The IceCube Neutrino Observatory is composed of many light detectors, called "DOMs" (digital optical modules), buried two kilometers deep below the surface of the South Pole. These light detectors are arranged in a roughly three dimensional grid: there are many "strings" hanging vertically downwards in the ice, and on each string there are many DOMs: 

![](../resources/images/I3det_v2_edited.jpeg)

When a high energy charged particle passes through the ice, it produces what is called Cherenkov light, which can then be detected by the many DOMs. The **locations** of the DOMs that see light, the **times** at which they saw light, and the **amount** of light they see all communicate information about this charged particle.

As experimentalists, what we want to do is work backwards: if what we have is this information about the light seen by the detector, what can we figure out about the original particle? 

In particular, if we want to do astrophysics like a telescope, what we really need to be able to do is to figure out the direction in which the original neutrino came from. 

# Part One: From what direction did the neutrino come? 

## 1: Visualizing events:

For this activity, we've prepared a simulation of many events within the IceCube detector. Execute the cell below by clicking the triangle or hitting `shift return` to download the simulation from google drive. Google CoLab will ask you if you trust the notebook: hit `Run Anyway`. 

In [ ]:
# needed for figures to appear in colab 
from google.colab import output
output.enable_custom_widget_manager()

!rm -rf sample_data
!mkdir sim_moonshadow

# download tracks.parquet
!gdown 1nqffw6xHLdX2oO8d-5xjExYZryo7rp3x
!mv tracks.parquet sim_moonshadow/tracks.parquet

# download cascades.parquet
!gdown 1yo3jD0a9xB2FfIXJJuMRc71IMe6nztvq
!mv cascades.parquet sim_moonshadow/cascades.parquet

# download pre-prepared analysis code
!rm -rf IceCube_MasterClass_at_Harvard2024
!git clone "https://github.com/kcarloni/IceCube_MasterClass_at_Harvard2024";

import sys
sys.path.insert(0, "./IceCube_MasterClass_at_Harvard2024/")

In [ ]:
from src.event_reader import load_sim_events

tracks = load_sim_events("sim_moonshadow/tracks.parquet")
tracks

Let's look at an 'event' in the detector. An event is just a bunch of 'hits', ie. instances in which light hit different sensors, which are grouped together because they all occured within the same time window.

In [ ]:
evt_num = 24
evt = tracks[evt_num]
evt

A list of hits is not that informative. We can *visualize* an event within the detector by plotting it.  

In [ ]:
from src.plot_event import *

fig = display_evt( evt )
fig.show()

In the event displays we've just looked at, redder hits occur earlier and greener hits occur after. Given this information, can you figure out in what direction the neutrino was going?

## 2: Neutrino directions:

Neutrinos travel towards our detector from every direction in the sky. We describe directions in the sky in terms of two angles, the *zenith* and the *azimuth*. 

The *zenith* describes the angle with respect to up, and takes values betwen 0 and 180. `zenith = 0` when the neutrino is traveling upwards, and `zenith = 180` when the neutrino is traveling down into the earth.  

*Azimuth* then describes the rotation of the travel direction around the up direction. We usually choose a reference point to be `azimuth = 0`. In IceCube, we use the Greenwich Meridian. 

If you've seen them before, these are just like the angles in spherical coordinates, θ, ϕ! 

![](../resources/images/zen_azi.png)

We've written a game below to see how well you can do. Execute the cell below (`shift return`) to start the game. 

You should see a similar event display, along with a big arrow, which represents your guess for the direction of the neutrino. Adjust this arrow using the zenith and azimuth angle sliders provided. Once you are happy with your guess, hit the submit button below the event display. The event display will be refreshed to show the actual true path of the neutrino, and how many degrees you were off the actual path will be printed below. Try to be as close as possible to the actual path, and minimize the degrees you were off by!

After submitting a guess, you can return to the game by clicking the Return button. You can then try to guess again on the same event, or try your hand at a different one by opening the event_id dropdown menu and selecting a different one.

In [ ]:
from src.reco_game import reco_game
reco_game( tracks )

## 3: Can we use computers to do a better job at figuring out the direction?

When we try to figure out the direction in the game above, what we're actually doing is trying to get the direction arrow to go as close to through the middle of all the hits as possible.

We can quantify this sense of "how good" with some math! Imagine drawing straight lines from each hit to the arrow, like in the image below:

![](../resources/images/linefit.png)

the more of these lines are shorter, the better our guess of the direction. We can calculate the length of each line using the Pythagorean theorem and some vector math: 

![](../resources/images/linefit_math.png)

We can also write a *function* in python to do this calculation for us! Just like in math, this *function* takes in some inputs (the direction vector and the hit) and produces one output (the perpendicular distance). 

(One detail we haven't mentioned is that in order to specify the direction arrow, we've also had to specify a `pivot_point`. This is just like defining a line by `y = mx + b` -- you need both a slope (the direction) and an intersection point (the pivot). See `calc_center_of_gravity` below for how we calculate the pivot! Can you understand how it works?)

In [ ]:
"""
This function uses the equation above to calculate the perpendicular distance between a single point and a pivot.
"""
def calc_perpendicular_distance( hit_pt, dir_vec, pivot_pt ):
    dist_vec = hit_pt - pivot_pt
    return np.sqrt( np.dot( dist_vec, dist_vec ) -  np.dot( dist_vec, dir_vec )**2 )


"""
This function calculates a pivot point given a set of hits.
"""
def calc_center_of_gravity( hits ):
    return hits.mean(axis=0)

Bad guesses should result in more lines being longer, ie. in a larger *mean* ( or average ) perpendicular distance. We can also calculate this with a function!

In [ ]:
from src.direction_utils import *

"""
Given a set of hits and a pair of guess angles, calculate the mean of all the perpendicular distances!
"""
def calc_mean_perpendicular_distance( dir_angles, hit_pts, pivot_pt ):
    
    dir_vec = get_direction_vector_from_angles( dir_angles[0], dir_angles[1] )
    return np.mean([ calc_perpendicular_distance(hit, dir_vec, pivot_pt) for hit in hit_pts ])

Let's check that bad guesses really result in a larger mean perpendicular distance. Adjust the values of the zenith `zen` and azimuth `azi` angles below and see how the mean distance changes. We'll also plot the direction vector again for a visual check on the 'goodness' of the guess.

Does the mean distance calculation help you refine your guess more easily? 

In [ ]:
from src.reco_game import reco_game
reco_game( tracks, calc_dist=True )

Just like you *minimized* the mean perpendicular distance by adjusting the zenith and azimuth values, a computer can minimize it using a minimization algorithm. Such algorithms define a procedure by which the computer repeatedly guesses some parameter (zenith and azimuth) values, evaluates the goodness of the guess (computes the mean distance), and then makes a new guess, over and over until some stopping condition (a condition that indicates that we think we've found the minimum).

The python `scipy` library, for scientific programming, offers many different minimization algorithms in its `scipy.optimize` module. Let's try using one! 

In [ ]:
from scipy.optimize import minimize


The `minimize` function takes in two inputs: 
- the first input is a function f whose input is a list of the parameters you want to minimize
- the second input is your initial guess for those parameters.

In [ ]:
def function_to_minimize( dir_angles ):
    return calc_mean_perpendicular_distance( dir_angles, evt.hits_xyz, pivot_pt )

def make_initial_guess_for_angles( evt ):

    # initial_guess_azi = np.deg2rad( -10 )
    # initial_guess_zen = np.deg2rad( 180 )
    # return np.array( [initial_guess_azi, initial_guess_zen] )
    
    j0, j1 = np.argmin( evt.hits_t ), np.argmax( evt.hits_t )
    if j0 == j1:
        initial_guess_azi = np.deg2rad( -10 )
        initial_guess_zen = np.deg2rad( 180 )
        return np.array( [initial_guess_azi, initial_guess_zen] )

    initial_guess = get_direction_angles_from_vector(
        normalize( evt.hits_xyz[j1, :] - evt.hits_xyz[j0, :] )
    )
    return initial_guess

Let's run the minimization and see how well it works!

In [ ]:
# change this if you want to try running it for different events: 
evt_num = 25
evt = tracks[evt_num]

# let's run the minimization!
minimization_output = minimize(
    function_to_minimize,
    make_initial_guess_for_angles( evt ),
)

best_guess_azi, best_guess_zen = minimization_output.x
smallest_distance = minimization_output.fun

# let's print the output: 
print( "The angles with the smallest perpendicular distance are:" )
print( f"azi = {np.rad2deg(best_guess_azi):.2f} degrees")
print( f"zen = {np.rad2deg(best_guess_zen):.2f} degrees")
print( "For these angles, the mean perpendicular distance is: \n")
print( f"\t {smallest_distance:.2f} meters")

# and let's plot the direction angle! 
dir_vec = get_direction_vector_from_angles( best_guess_azi, best_guess_zen )

fig = display_evt( evt )
fig.add_traces( plot_direction( dir_vec, pivot_pt, color="hotpink" ) )
fig.show()

<!-- Take a look at the plotted arrow. Is it going in the right direction?  -->

Let's also check the performance of our computer against the true answer.

(We can do this for these events because they are all simulated, so we know their true properties. If we were doing this with a real event from data collected by IceCube, we would not have any way of knowing this real right answer. This is why simulation is so important -- it gives us a way of checking how good we are at guessing the true quantities. )

In [ ]:
true_dir_vec = get_direction_vector_from_angles( 
    evt.true_muon_azimuth, 
    evt.true_muon_zenith )

fig.add_traces( plot_direction( true_dir_vec, pivot_pt, color="black" ) )
fig.show()

## 4. How can we quantify the quality of our "reconstructed" directions?

Let's say we have more than one method to determine the direction of an event. How can we decide which one is better? 

If you haven't yet, go back to step 3. and try changing which event you figure out the direction for. 
- Are there any kinds of events which are easier / harder for you to guess? 
- Are there any kinds of events which are easier / harder for the computer algorithm to reconstruct? 

As you may see, how well we do at figuring out the direction varies a lot between different events. If we want to pick the best method, we need to look at how well it does for many different events.

Let's start by using the computer algorithm to reconstruct the directions of many events. To do this fast, we'll use the code from above to write a function whose input is an event and whose output is the best guess direction. 

In [ ]:
def find_best_dir( evt ):

    pivot_pt = calc_center_of_gravity( evt.hits_xyz )
    
    # you can define functions inside of other functions!
    def function_to_minimize( dir_angles ):
        return calc_mean_perpendicular_distance( dir_angles, evt.hits_xyz, pivot_pt )

    initial_guess = make_initial_guess_for_angles( evt )

    out = minimize( 
        function_to_minimize,
        initial_guess, 
        method='Nelder-Mead'
    )

    best_azi, best_zen = out.x
    best_azi = bound_azi( best_azi )
    best_zen = bound_zen( best_zen )

    return np.array([best_azi, best_zen])

Let's use this function to reconstruct the directions of a bunch of events. 

In [ ]:
# change this number to adjust how many events you want to use! 
N = 100

# we create empty arrays to hold the reconstructed angles.
reco_azi = np.empty( N )
reco_zen = np.empty( N )

# let's also save the true angles. 
true_azi = np.empty( N )
true_zen = np.empty( N )

# now we iterate over all the events...
for evt_num in range(N):
    
    evt = tracks[evt_num]
    reco_azi[evt_num], reco_zen[evt_num] = find_best_dir( evt )

    true_azi[evt_num] = evt.true_muon_azimuth
    true_zen[evt_num] = evt.true_muon_zenith


Now what we need to calculate is the difference between the directions:

![](../resources/images/great_circle_distance.jpg)

The difference between two directions is given by some more vector math: see the function below. If you've worked with vectors or spherical coordinates before, see if you can figure out how this equation is derived! 

In [ ]:
def great_circle_distance( azi_1, zen_1, azi_2, zen_2  ): 

    dot_prod = \
        np.cos(zen_1) * np.cos(zen_2) + \
        np.sin(zen_1) * np.sin(zen_2) * np.cos( azi_1 - azi_2 )
        
    return np.arccos(dot_prod)

Let's figure out how big this difference angle is on average:

In [ ]:
diff_angle = great_circle_distance( true_azi, true_zen, reco_azi, reco_zen )

print("the average difference between \n")
print(f"\t the true and reconstructed directions is {np.rad2deg(diff_angle).mean():.2f}°" )

Is this what you expected?

#### Bonus: We can investigate more deeply by plotting a histogram...

Ask a researcher for help with this if you like!

# Part Two: If we know where the neutrinos came from, we can do astrophysics!

As we discussed this morning, showers of high energy protons called "cosmic rays" rain down on our solar system from outer space all the time, from every direction. When these cosmic rays hit the dense air in our atmosphere, they interact, producing many other long lived particles. Our detectors have been observing these for 100 years!

We also saw this morning that a thick material (like a sheet of plastic) can block many of these particles. Because the moon is close to us and very very dense, it can block cosmic rays from reaching our detectors! 

In other words, by looking for a *reduction* in the number of events in a particular direction, we can "see" the moon. This was the first kind of astrophysics IceCube could do!

(If you're interested in reading more about this, you can take a look at this press release: https://icecube.wisc.edu/news/research/2013/05/cosmic-ray-moon-shadow-seen-by-icecube/ or read the original publication https://arxiv.org/pdf/1305.6811!)

The dataset we've been looking at thus far is actually a simulation of this effect. We've also provided you with a bunch of reconstructed directions (using a more complicated / better algorithm than the one we wrote above). 

In [ ]:
from src.event_reader import load_sim_events
tracks = load_sim_events( "sim_moonshadow/tracks.parquet" )

## 1: Where is the moon in the sky? 

In [ ]:
# software to work with astronomomical objects
!git clone https://github.com/jlazar17/pyorbital
sys.path.insert(0, "./pyorbital/")

!python3 -m pip install healpy
import numpy as np

Think about the last time you looked at the moon. Where was it, in the sky? Was it directly overhead? 

The position of the moon in the sky of any observer on the surface of the earth depends on the observer's time, date and location. Because the earth is round, we specify the observer's location on the earth using two angles, called `latitude` and `longitude`. (Exactly like how we use two angles to describe the neutrino's direction!)

![](../resources/images/lat_long.png)

We've written a function to calculate the position of the moon at any given day, time, and location. Where will the moon be tonight in Cambridge at 7pm? 

In [ ]:
from src.jdutil import mjd_to_datetime
from src.moon import get_moon_position_at

# in degrees
cambridge_lat = 42.37
cambridge_long = 71.11

date_and_time = "2024-04-27-19-00-00" # Today at noon !

moon_azi, moon_zen = get_moon_position_at(
    date_and_time, cambridge_lat, cambridge_long)

print(
    f"at {date_and_time}, in Cambridge, \n",
    f"the moon was at azimuth = {moon_azi:.2f}°, zenith = {moon_zen:.2f}°"
)

From  the picture above, can you figure out at what latitude is the IceCube Observatory? Fill it out below!

( If you're curious about other locations, or want to check your work, you can use this website https://getlatlong.net/ to find the latitudes and longitudes of other locations on the earth. )

In [ ]:
icecube_lat = # fill me out! 
icecube_long = 45.

In order to figure out if a particular event was near the moon or not, we need to check where in the South Pole sky the moon was at that time! Let's do this for some events in our sample. 

In [ ]:
evt_num = 24
evt = tracks[evt_num]

# this date + time is written as a "Modified Julian Date"
evt_date_and_time = evt.mjd_time

moon_azi, moon_zen = get_moon_position_at(
    evt_date_and_time, icecube_lat, icecube_long)

print(
    f"at {mjd_to_datetime(evt_date_and_time)}, at the South Pole, \n",
    f"the moon was at azimuth = {moon_azi:.2f}°, zenith = {moon_zen:.2f}°"
)

## 2: The shadow of the moon

Okay, let's get down to business. Now we actually want to calculate 
- where the moon was at the day + time of each event
- how far away the moon's position in the sky was from the direction the event came in. 

Let's take a look at our event sample again. How many years of data have we given you? 

In [ ]:
# let's get the reconstructed direction of each event: 
evt_azimuths = np.rad2deg( tracks["reco_azimuth"] )
evt_zeniths = np.rad2deg( tracks["reco_zenith"] )

# and the time: 
evt_dates = tracks["mjd_time"]

start_date = mjd_to_datetime( np.min( tracks["mjd_time"] ) )
end_date = mjd_to_datetime( np.max( tracks["mjd_time"] ) )
start_date, end_date

In [ ]:
# Now we iterate over all the dates, and calculate the moon position for each.
# At each step, we will `append` the zenith and azimuth to our lists

# Heads up - this step takes ~30 seconds!
moon_zeniths = []
moon_azimuths = []

for date in evt_dates:
    azi, zen = get_moon_position_at( date, icecube_lat, icecube_long )
    moon_azimuths.append(azi)
    moon_zeniths.append(zen)

We now need to calculate the differences between the two sets of angles. (We'll do these separately so we can plot the differences in two dimensions). We've written some functions to do this!

Can you figure out what's going on in the azimuth difference calculation? 

In [ ]:
def calc_zenith_diff( zen_1, zen_2 ):
    return zen_1 - zen_2

def calc_azimuth_diff( azi_1, azi_2 ):
    # this is a little trickier, because 0° and 360° (or, in radians, 0 and 2pi) mean the same thing!
    # we need to remember that the biggest possible difference between two angles is 180° (pi radians).
    diff =  azi_1 - azi_2
    return [ diff, diff-360, diff+360 ][ 
        np.argmin( [abs(diff), abs(360 - diff), abs(360 + diff)] )
    ]

Let's calculate the differences and make a plot!

In [ ]:
diff_zeniths = calc_zenith_diff( moon_zeniths, evt_zeniths )
diff_azimuths =  [ calc_azimuth_diff( a1, a2 ) for (a1, a2) in zip( moon_azimuths, evt_azimuths ) ]

from src.plot_hist import plot_heatmap

minimum_angle = -5 # degrees
maximum_angle = 5 # degrees
n_bins = 10

angle_bins = np.linspace(minimum_angle, maximum_angle, n_bins+1)

h, _, _ = np.histogram2d(
    np.sin( np.deg2rad(evt_zeniths) ) * diff_azimuths,
    diff_zeniths,
    bins=[angle_bins, angle_bins]
)

plot_heatmap(h, angle_bins, cmap="Greys_r")

#### Bonus:

How well we can see the moon depends on our *resolution*, roughly how (not) blurry our sight with the detector is -- in other words, how good we are at reconstructing the correct direction of the event from the hits we see in our detector!

This reconstruction ability generally depends on the *energy* of the particle making the event. Can you figure out why? 

The vector below contains the energies of the initial particles causing each of the events. If you're familiar with python at all, try:
- isolating the events with higher or lower energy
- checking how good the reconstruction in angle is, on average
- and checking what the moon shadow looks like using only those events.

Feel free also to ask one of the researchers for help! 

In [ ]:
evt_energy = tracks["initial_state_energy"]

true_zenith = tracks["initial_state_zenith"]
true_azimuth = tracks["initial_state_azimuth"]

# Bonus preview activity: cascades...

As we will discuss later, IceCube can see two broad kinds of events, *tracks*, which look like long lines and are produced by the light from *muons*, and *cascades*, which look like spherical blobs and are produced by the light from *electrons* (and some other particles). 

We just tried to reconstruct the direction of some track events. Do you think it would be easier or harder to reconstruct the direction of some cascade events? 

Give it a try!

In [ ]:
cascades = load_sim_events( "sim_moonshadow/cascades.parquet" )
reco_game( cascades, event_type="cascade" )